# Agentic RAG

**Agentic RAG (Retrieval-Augmented Generation with Agents)** is an advanced evolution of RAG that adds autonomy, reasoning, and multi-step decision-making to the retrieval process.

Instead of passively fetching chunks from a knowledge base and sending them to an LLM, Agentic RAG allows an intelligent agent to decide:

- What information to retrieve

- Where to retrieve it from (DB, API, or documents)

- When retrieval is unnecessary

- How to verify or refine the retrieved knowledge before answering

In simple terms, the **LLM acts as a decision-making agent**, not just a passive responder.

### Use Case 
Imagine we’re building a **Medical Assistant Chatbot** for a hospital.

Chroma Retriever → Trained on structured policies, SOPs, and hospital manuals

FAISS Retriever → Trained on unstructured patient Q&A or case descriptions

API Retriever → Calls a live API (like CDC or WHO) for up-to-date disease info.

When a doctor types:

“What is the isolation protocol for a patient with suspected MERS?”

The Agentic RAG workflow could be:

- The agent checks intent: “This is about hospital protocol.”

- Uses Chroma Retriever to fetch official policy.

- Cross-checks medical context with FAISS Retriever for case handling advice.

- Optionally queries WHO API for the latest recommendations.

- Synthesizes an answer using the LLM.

In [24]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import Document
import requests

In [5]:
# Initialize embeddings and model
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
# Example structured data (Hospital SOPs)
sop_texts = [
    "All patients suspected of having MERS must be isolated immediately in negative pressure rooms.",
    "Doctors must wear N95 masks and PPE kits when handling contagious cases.",
    "All waste from infectious wards should be disposed of as per biohazard protocol section 3.2."
]

# Example unstructured Q&A data (Semantic)
qa_texts = [
    "A patient with MERS showed symptoms of fever, cough, and difficulty breathing. He was isolated within 2 hours.",
    "If the hospital is full, temporary isolation using portable barriers is acceptable for suspected infections.",
    "Patients with COVID-19 and MERS require separate wards due to cross-contamination risks."
]

# Convert to LangChain Document objects
sop_docs = [Document(page_content=t) for t in sop_texts]
qa_docs = [Document(page_content=t) for t in qa_texts]


In [14]:
# Create in-memory vector stores
sop_faiss_db = FAISS.from_documents(sop_docs, embedding=embeddings)
qa_faiss_db = FAISS.from_documents(qa_docs, embedding=embeddings)

print("✅ In-memory retrievers initialized successfully.")

✅ In-memory retrievers initialized successfully.


In [15]:
# Convert to retrievers
sop_retriever = sop_faiss_db.as_retriever(search_kwargs={"k": 2})
qa_retriever = qa_faiss_db.as_retriever(search_kwargs={"k": 2})

In [16]:
def api_retriever(query: str):
    """Mock retriever for external data (WHO API example)."""
    if "covid" not in query.lower():
        return "No live data relevant to this query."
    try:
        response = requests.get("https://disease.sh/v3/covid-19/all", timeout=5)
        data = response.json()
        return f"Live API data: {data['cases']:,} total COVID cases globally, {data['deaths']:,} deaths."
    except Exception as e:
        return f"API fetch failed: {e}"

#### Agentic Decision Logic

In [22]:
from langchain.agents import Tool
from langchain.schema import HumanMessage, SystemMessage

def create_retrieval_tools():
    """Create tools for different retrieval sources."""
    def sop_search(query: str) -> str:
        docs = sop_retriever.get_relevant_documents(query)
        return "\n".join([doc.page_content for doc in docs])
    
    def qa_search(query: str) -> str:
        docs = qa_retriever.get_relevant_documents(query)
        return "\n".join([doc.page_content for doc in docs])
    
    def api_search(query: str) -> str:
        return api_retriever(query)
    
    return [
        Tool(name="sop_search", description="Search hospital SOPs and policies", func=sop_search),
        Tool(name="qa_search", description="Search patient Q&A and case studies", func=qa_search),
        Tool(name="api_search", description="Get live data from external APIs", func=api_search)
    ]

tools = create_retrieval_tools()

In [25]:
class AgenticRAG:
    def __init__(self, llm, tools):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
        
    def _decide_retrieval_strategy(self, query: str) -> list:
        """LLM decides which retrieval sources to use."""
        print(f"🤔 Analyzing query: '{query}'")
        print("📋 Available sources:")
        for tool_name, tool in self.tools.items():
            print(f"  - {tool_name}: {tool.description}")
        
        system_prompt = """You are a medical assistant. Analyze the query and decide which retrieval sources to use.
        
        Available sources:
        - sop_search: Hospital policies, SOPs, official procedures
        - qa_search: Patient cases, Q&A, practical examples  
        - api_search: Live data, current statistics

        Return only the tool names as a comma-separated list. Example: "sop_search,qa_search"
        """
        
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"Query: {query}")
        ]
        
        response = self.llm.invoke(messages)
        tool_names = [name.strip() for name in response.content.split(",")]
        selected_tools = [name for name in tool_names if name in self.tools]
        
        print(f"🎯 LLM selected sources: {selected_tools}")
        return selected_tools
    
    def _retrieve_and_synthesize(self, query: str, tool_names: list) -> str:
        """Retrieve from selected sources and synthesize response."""
        print(f"\n🔍 Retrieving information from {len(tool_names)} sources...")
        retrieved_info = []
        
        for tool_name in tool_names:
            print(f"  📖 Querying {tool_name}...")
            tool = self.tools[tool_name]
            result = tool.func(query)
            retrieved_info.append(f"From {tool_name}: {result}")
            print(f"    ✅ Retrieved {len(result)} characters")
        
        print(f"\n🧠 Synthesizing response...")
        context = "\n\n".join(retrieved_info)
        
        synthesis_prompt = f"""Based on the retrieved information, provide a comprehensive answer to the query.

        Query: {query}

        Retrieved Information:
        {context}

        Answer:"""
        
        response = self.llm.invoke([HumanMessage(content=synthesis_prompt)])
        print("✅ Response generated")
        return response.content
    
    def query(self, question: str) -> str:
        """Main query method that orchestrates the agentic RAG process."""
        print(f"\n🚀 Starting Agentic RAG process...")
        tool_names = self._decide_retrieval_strategy(question)
        return self._retrieve_and_synthesize(question, tool_names)

# Initialize Agentic RAG
agentic_rag = AgenticRAG(llm, tools)


In [26]:
# Test Agentic RAG
question = "What is the isolation protocol for a patient with suspected MERS?"
answer = agentic_rag.query(question)
print(f"Question: {question}")
print(f"Answer: {answer}")


🚀 Starting Agentic RAG process...
🤔 Analyzing query: 'What is the isolation protocol for a patient with suspected MERS?'
📋 Available sources:
  - sop_search: Search hospital SOPs and policies
  - qa_search: Search patient Q&A and case studies
  - api_search: Get live data from external APIs
🎯 LLM selected sources: ['sop_search']

🔍 Retrieving information from 1 sources...
  📖 Querying sop_search...
    ✅ Retrieved 167 characters

🧠 Synthesizing response...
✅ Response generated
Question: What is the isolation protocol for a patient with suspected MERS?
Answer: The isolation protocol for a patient with suspected Middle East Respiratory Syndrome (MERS) requires immediate action to prevent the transmission of the virus. Here are the key components of the protocol:

1. **Immediate Isolation**: Patients suspected of having MERS must be isolated immediately upon suspicion of infection. This is crucial for containing the virus and ensuring the safety of other patients and healthcare workers.


In [27]:
question = "What are covid stats so far?"
answer = agentic_rag.query(question)
print(f"Question: {question}")
print(f"Answer: {answer}")


🚀 Starting Agentic RAG process...
🤔 Analyzing query: 'What are covid stats so far?'
📋 Available sources:
  - sop_search: Search hospital SOPs and policies
  - qa_search: Search patient Q&A and case studies
  - api_search: Get live data from external APIs
🎯 LLM selected sources: ['api_search']

🔍 Retrieving information from 1 sources...
  📖 Querying api_search...
    ✅ Retrieved 72 characters

🧠 Synthesizing response...
✅ Response generated
Question: What are covid stats so far?
Answer: As of the latest live data available, there have been a total of 704,753,890 confirmed COVID-19 cases worldwide. The pandemic has resulted in approximately 7,010,681 deaths globally. These statistics reflect the ongoing impact of COVID-19 across various nations, highlighting the importance of continued public health efforts and vaccination campaigns to combat the virus.


In [28]:
question = "What are the symptomps of MERS and what should be the initial steps by doctors?"
answer = agentic_rag.query(question)
print(f"Question: {question}")
print(f"Answer: {answer}")


🚀 Starting Agentic RAG process...
🤔 Analyzing query: 'What are the symptomps of MERS and what should be the initial steps by doctors?'
📋 Available sources:
  - sop_search: Search hospital SOPs and policies
  - qa_search: Search patient Q&A and case studies
  - api_search: Get live data from external APIs
🎯 LLM selected sources: ['qa_search', 'sop_search']

🔍 Retrieving information from 2 sources...
  📖 Querying qa_search...
    ✅ Retrieved 199 characters
  📖 Querying sop_search...
    ✅ Retrieved 167 characters

🧠 Synthesizing response...
✅ Response generated
Question: What are the symptomps of MERS and what should be the initial steps by doctors?
Answer: MERS, or Middle East Respiratory Syndrome, presents a range of symptoms primarily affecting the respiratory system. The key symptoms to look out for include:

1. **Fever** - Often one of the first indicators.
2. **Cough** - May develop as the illness progresses.
3. **Difficulty Breathing** - This can range from mild to severe and is 